In [1]:
import camelot
import numpy as np
import pandas as pd
import json
from stuff import output_directory, tablestocellinfo, file, filename, cleanuptables, is_first_page, get_first_date_cell, get_num_pages, get_pdf_size
import os
import sys

from pdf2image import convert_from_path
pdfs = convert_from_path(file)

h,w = get_pdf_size(file)
new_h,new_w = pdfs[0].height, pdfs[0].width

def get_color(img,cell):
    y = new_h - ((cell.y1 + 3)/h * new_h)
    x = ((cell.x1 + cell.x2)/2)/w * new_w
    return img.getpixel((x,y))

tables = camelot.read_pdf(file,pages='1-end',flavor='lattice',line_scale=20)
colors = set()
for idx, table in enumerate(tables):
    for row in table.cells:
        for cell in row:
            colors.add(get_color(pdfs[idx],cell))
#colors = (255, 255, 0), (198, 198, 198), (240, 240, 240), (255, 128, 255), (0, 128, 0), (255, 0, 0), (129, 129, 129), (122, 122, 122), (128, 0, 255), (255, 255, 255), (80, 138, 160), (128, 128, 64), (128, 128, 128), (128, 0, 64)

In [ ]:
def old_main():
    parts = []
    for offset in range(0,len(tables),get_num_pages(tables)):
        first_page = tables[0+offset].df.iloc[1:,1:].copy()
        first_page.iloc[0,0] = 'Starfsmaður'
        second_page = tables[1+offset].df.copy()
        third_page = tables[2+offset].df.copy()

        first_page.index -= 1
        first_page.columns -= 1
        # second_page.index -= 1
        # third_page.index -= 1


        x,y = get_first_date_cell(second_page)
        first_page.iloc[0,1:] = second_page.iloc[y,x:]

        second_page = second_page.iloc[y+1:,x-1:]
        second_page.index -= 1 
        second_page.columns -= 1

        x,y = get_first_date_cell(third_page)
        third_page = third_page.iloc[y+1:,x-1:]
        third_page.index -= 1
        third_page.columns -= 1

        joined = pd.concat([first_page,second_page,third_page],ignore_index = True)
        joined.iloc[:,:]#.to_csv(os.path.join(output_directory,'out' + str(offset) + '.csv'))
        parts.append(joined)
    total = pd.concat(parts,axis=1,ignore_index=True)
    total.to_csv(os.path.join(output_directory,'output.csv'))



parts = []
def new_main():
    for offset in range(0,len(tables),get_num_pages(tables)):
        pgs = []
        for page_idx in range(offset,offset+get_num_pages(tables),1):
            df = tables[page_idx].df.copy()
            x,y = get_first_date_cell(df)
            df.columns = df.iloc[y,:]
            # df = df.iloc[y+1:,x-1:]
            pgs.append(df)

        # first_page = tables[0+offset].df.copy()
        # second_page = tables[1+offset].df.copy()
        # third_page = tables[2+offset].df.copy()

        # x,y = get_first_date_cell(first_page)
        # first_page.columns = first_page.iloc[y,:]
        # first_page = first_page.iloc[y+1:,x-1:]
        # first_page.reset_index(inplace=True, drop=True)

        # x,y = get_first_date_cell(second_page)
        # second_page.columns = second_page.iloc[y,:]
        # second_page = second_page.iloc[y+1:,x-1:]
        # second_page.reset_index(inplace=True, drop=True)

        # x,y = get_first_date_cell(third_page)
        # third_page.columns = third_page.iloc[y,:]
        # third_page = third_page.iloc[y+1:,x-1:]
        # # second_page = second_page.iloc[y+1:,x-1:]
        # third_page.reset_index(inplace=True, drop=True)


        # x,y = get_first_date_cell(third_page)
        # third_page = third_page.iloc[y+1:,x-1:]


        joined = pd.concat(pgs,axis=0)
        parts.append(joined)
    # total = pd.concat(parts,axis=1,ignore_index=True)
    # total.to_csv(os.path.join(output_directory,'output.csv'))

new_main()

In [4]:
pgs = []
for offset in range(0,len(tables),get_num_pages(tables)):
    print(offset)
    for page_idx in range(offset,offset+get_num_pages(tables),1):
        df = tables[page_idx].df.copy()
        print(df)
        x,y = get_first_date_cell(df)
        df.columns = df.iloc[y,:]
        df = df.iloc[y+1:,x-1:]
        pgs.append(df)

#     # first_page = tables[0+offset].df.copy()
#     # second_page = tables[1+offset].df.copy()
#     # third_page = tables[2+offset].df.copy()

#     # x,y = get_first_date_cell(first_page)
#     # first_page.columns = first_page.iloc[y,:]
#     # first_page = first_page.iloc[y+1:,x-1:]
#     # first_page.reset_index(inplace=True, drop=True)

#     # x,y = get_first_date_cell(second_page)
#     # second_page.columns = second_page.iloc[y,:]
#     # second_page = second_page.iloc[y+1:,x-1:]
#     # second_page.reset_index(inplace=True, drop=True)

#     # x,y = get_first_date_cell(third_page)
#     # third_page.columns = third_page.iloc[y,:]
#     # third_page = third_page.iloc[y+1:,x-1:]
#     # # second_page = second_page.iloc[y+1:,x-1:]
#     # third_page.reset_index(inplace=True, drop=True)


#     # x,y = get_first_date_cell(third_page)
#     # third_page = third_page.iloc[y+1:,x-1:]


#     joined = pd.concat(pgs,axis=0)
#     parts.append(joined)
# # total = pd.concat(parts,axis=1,ignore_index=True)
# # total.to_csv(os.path.join(output_directory,'output.csv'))


0
                                 0                        1            2   \
0                    t\nStarfsmaður                            11.08\nfim   
1   f\nHæfniþáttur : VEL.stuðningur                                         
2                                       Agnes Rut Ásgeirs...                
3                                               Arndís prufa                
4                                     Aron Freyr Kristján...                
5                                      Benedikt Pétur Sig...        ORLOF   
6                                        Björg Hákonardóttir  16:00-22:00   
7                                         Breki Benediktsson                
8                                           Daníel Pétursson                
9                                      Davíð Arnljótur Ka...  08:00-16:00   
10                                     Friðberg Reynir Tr...        ORLOF   
11                                      Guðbjartur Máni G...              

In [21]:
pd.concat([pgs[0],pgs[1],pgs[2]])

,,11.08\nfim,12.08\nfös,13.08\nlau,14.08\nsun,15.08\nmán,16.08\nþri,17.08\nmið,18.08\nfim,19.08\nfös,20.08\nlau,21.08\nsun,22.08\nmán,23.08\nþri,24.08\nmið
1,,,,,,,,,,,,,,,
2,Agnes Rut Ásgeirs...,,16:00-23:00,08:00-16:00,08:00-16:00,,,,15:00-23:00,,,,,,
3,Arndís prufa,,15:00-23:00,16:00-23:00,16:00-23:00,,,,12:00-23:00,,,,16:00-23:00,,
4,Aron Freyr Kristján...,,,,,08:00-16:00,09:00-20:00,,09:00-20:00,,08:00-16:00,08:00-16:00,23:00-09:00,,16:00-23:00
5,Benedikt Pétur Sig...,ORLOF,ORLOF,,,ORLOF,ORLOF,ORLOF,ORLOF,ORLOF,,,ORLOF,,
6,Björg Hákonardóttir,16:00-22:00,,,,16:00-22:00,,,,,08:00-16:00,08:00-16:00,,15:00-22:00,
7,Breki Benediktsson,,16:00-22:00,16:00-22:00,16:00-22:00,,,09:00-20:00,,,,,09:00-20:00,,
8,Daníel Pétursson,,09:00-20:00,16:00-23:00,16:00-23:00,16:00-23:00,,,,,,,,,
9,Davíð Arnljótur Ka...,08:00-16:00,08:00-16:00,,,09:00-20:00,,08:00-16:00,,16:00-22:00,08:00-16:00,08:00-16:00,,,08:00-16:00
10,Friðberg Reynir Tr...,ORLOF,ORLOF,,,ORLOF,ORLOF,ORLOF,ORLOF,ORLOF,,,ORLOF,ORLOF,ORLOF
